# Contributions
```{glue:figure} location_components
:scale: 25%
:align: right
```

In this notebook, we'll examine the various contributions to extreme sea levels through the lens of the nonstationary GEV analysis. Inspiration for these plots and analysis follows from {cite:t}`marra_advancing_2023` and {cite:t}`sweet_implications_2024`, which use the nonstationary GEV analysis of {cite:t}`mendez_analyzing_2007` to analyze the influence of climatic patterns like ENSO and PDO (for example) on extreme sea levels. This notebook changes only one aspect of the method - allowing for a lagged covariate in the nonstationary GEV model.  

NOTE: Please view this notebook as experimental until we get the kinks out.

## Setup

First we'll import the necessary functions, set up our directories, and get everything ready for plotting. This should be the same as we did previously in the [nonstationary extremes](notebooks/nonstationaryGEV/monthly_extremes_non-stationaryGEV.ipynb) notebook.

In [1]:
%run ../0_1_setup.ipynb
import sys
import json
from scipy.stats import chi2
sys.path.append("../../python/nonstationaryGEV")
import helpers, models, plotting, imports

base_dir = Path(data_dir).parent
dirs = imports.make_directoryDict(base_dir)
model_output_dir = dirs['model_output_dir']
from helpers import adjust_w_for_plotting


Using custom paths from config_env.py
Data directory: /Users/jfiedler/Projects/SL_Indicators/data
Output directory: /Users/jfiedler/Projects/SL_Indicators/output
Using custom paths from config_env.py
Data directory: /Users/jfiedler/Projects/SL_Indicators/data
Output directory: /Users/jfiedler/Projects/SL_Indicators/output


## Compile data 
```{caution}

Before running this, you will need to run the [non-stationary models](notebooks/nonstationaryGEV/monthly_extremes_non-stationaryGEV.ipynb) notebook first! Grab some coffee and get ready for some fun.
```

In the following codes we are just compiling the output and results from those models that have been saved to "model_output_dir" directory. The methodology contained herein follows {cite:t}`sweet_implications_2024`.

In [2]:
rsl_hourly = xr.open_dataset(data_dir/ 'rsl_hawaii_noaa.nc')
rsl_hourly['station_name'] = rsl_hourly['station_name'].astype(str)

station_ids = rsl_hourly.station_id.values
runWithoutModel = True
returnPeriod = [2,10,50,100]
year0plot = 1993
saveToFile = True
numProcesses = 8 # number of processes to run in parallel, select 1 if you want to run in serial
climateIndex = ['PMM','BEST','ONI','PDO','AO','PNA','TNA'] #add more, if needed


Establish directories if you have not already done so. (They should be already established if you've run the [non-stationary models](notebooks/nonstationaryGEV/monthly_extremes_non-stationaryGEV.ipynb) notebook already. Have you done that? You should. It'll be fun, try it.)

In [3]:
# check if output directory exists, if not create it
if not output_dir.exists():
    output_dir.mkdir(parents=True)

# check if model output directory exists, if not create it
if not dirs['model_output_dir'].exists():
    dirs['model_output_dir'].mkdir(parents=True)

# make sub-directories for each station in rsl_hourly in model_output_dir
for sid in rsl_hourly.station_id:
    sidString = str(sid.values)
    # maybe needs some leading zeros, for now we'll leave it.
    station_dir = dirs['model_output_dir'] / sidString
    if not station_dir.exists():
        station_dir.mkdir(parents=True)

The following section will run the covariate in scale and location model for every climate covariate in the covariateIndex variable. Only run this section 1 time. 

In [5]:
for stationID in station_ids:
    print(f'Processing stationID: {stationID}')
    #only run if best_params.json does not exist
    best_params_file = dirs['model_output_dir'] / str(stationID) / 'best_params.json'
    if best_params_file.exists():
        print(f'Best parameters file already exists for stationID {stationID}, skipping...')
        continue

    # Preallocate the significance array
    SignifCvte1 = np.zeros(len(climateIndex))
    SignifCvte2_loc = np.zeros(len(climateIndex))
    SignifCvte2_T = np.zeros(len(climateIndex))

    runWithoutModel=True
    numProcesses = 8
    # run the models for all recordIDs
    # for recordID in recordIDs:
    _, _, _, _, _, _, x_N, w_N, wcomp, SignifN = models.run_noClimateIndex_models(rsl_hourly,stationID,runWithoutModel,dirs, returnPeriod, CIname='None', nproc=numProcesses)

    # you can set runWithoutModel=False if you want to re-run the models
    STNDtoMHHW, station_name, year0, mm, ampCvte1, SignifCvte1 = models.run_CI_models(rsl_hourly,stationID,runWithoutModel,dirs, returnPeriod, climateIndex,x_N, w_N, wcomp, SignifN, nproc=numProcesses)


Processing stationID: 1617760
Best parameters file already exists for stationID 1617760, skipping...
Processing stationID: 1619910
Best parameters file already exists for stationID 1619910, skipping...
Processing stationID: 1617433
Best parameters file already exists for stationID 1617433, skipping...
Processing stationID: 1612480

******Running models without climate index...******
Data prepared for model input in  /Users/jfiedler/Projects/SL_Indicators/data/model_run
Statistical Significance of Linear Trend: 100.00%
Estimated Trend on monthly Maxima values is: 2.88 +/- 0.38 mm/year
Model already saved to netcdf file.
The trend is significant! 
Include long-term trend and nodal cycle in final model.
Statistical Significance of adding Nodal cycle: 99.94%
Model exists:  /Users/jfiedler/Projects/SL_Indicators/data/GEV_model_output/1612480/RL_muN.nc

****Running models with climate index...****
Data prepared for model input in  /Users/jfiedler/Projects/SL_Indicators/data/model_run
Statist

RuntimeError: MPI execution failed.

#### Choose station and climate indices
Here we'll use all tested indices

In [ ]:
climateIndex = ['AO','BEST','ONI','PDO','PMM','PNA','TNA']

# get list of all directories in dirs['model_output_dir']
dirs_list = os.listdir(dirs['model_output_dir'])

# only keep the directories that are numbers
stationIDs = [int(d) for d in dirs_list if d.isdigit()]


# get station names from the best_params.json files
station_names = {}
for stationID in stationIDs:
    jsonpath = Path(dirs['model_output_dir']) / str(stationID) / 'best_params.json'
    with open(jsonpath, 'r') as f:
        output = json.load(f)
        station_names[stationID] = output['modelInfo']['station_name']

# sort the station names by the stationID
station_names = {k: v for k, v in sorted(station_names.items(), key=lambda item: item[0])}

station_names

#### Create a dataframe
This will hold information about the amplitude of the tested climate index signals in the location parameter of the best-fit nonstationary GEV model.

In [ ]:
#%%
for stationID in station_ids:
    # Initialize an empty list to store results
    results = []
    SignifCvte1 = np.zeros(len(climateIndex))
    w_nodal = np.zeros(len(climateIndex))
    w_cvte1 = np.zeros(len(climateIndex))

    for i in np.arange(0, len(climateIndex)):
        #read first value of w from trend_params.json
        nodal_params = 'nodal_params.json'
        jsonpath = Path(dirs['model_output_dir']) / str(stationID) / nodal_params
        with open(jsonpath, 'r') as f:
            output = json.load(f)
            w_nodal[i] = output['w'][0]

        covariate_params = f'cvte_location_params_{climateIndex[i]}.json'

        # Create the full path for the JSON file
        jsonpath = Path(dirs['model_output_dir']) / str(stationID) / covariate_params

        # Open and read the JSON file
        with open(jsonpath, 'r') as f:
            output = json.load(f)
        w, mio, standard_error,x = (np.array(output[key]) for key in ['w', 'mio', 'standard_error','x'])

        wfull = helpers.adjust_w_for_plotting(x,w)

        # prepend standard error with 0 to match the size of w
        standard_error = np.insert(standard_error, 0, 0)

        standard_error = helpers.adjust_w_for_plotting(x,standard_error)

        # Store the results in a list
        results.append({
            'Climate Index': climateIndex[i],
            'CI param': wfull[10],  
            'Standard Error of CI param': standard_error[10]
        })

        # Compute deviance statistic
        w_cvte1[i] = w[0]
        diffe = w_cvte1[i] - w_nodal[i]
        p = 1
        SignifCvte1[i] = chi2.cdf(2 * diffe, p)

    # Convert the results list to a DataFrame
    df_cvteLocation = pd.DataFrame(results)

    # add Significance to the dataframe
    df_cvteLocation['Significance (over Nodal)'] = SignifCvte1


    # add the lags from CI_correlation_results.csv
    CI_correlation_results = pd.read_csv(Path(dirs['CI_dir']) / 'CI_correlation_results_setLag.csv')

    #rename climateIndex to Climate Index
    CI_correlation_results = CI_correlation_results.rename(columns={'climateIndex':'Climate Index'})

    # extract only the stationID of interest
    CI_corr = CI_correlation_results[CI_correlation_results['stationID'].astype(str) == str(stationID)]
    # add the lags from CI_correlation_results['lag'] to df_cvteLocation, matching on Climate Index and stationID
    df_cvteLocation = df_cvteLocation.merge(CI_corr[['Climate Index','lag']], on='Climate Index')


    #save to a json file
    df_cvteLocation.to_json(Path(dirs['model_output_dir']) / str(stationID) / 'cvte_location_params_ALL.json')
    df_cvteLocation


### Investigate the parameter magnitudes and lags

First let's take a look at an example dataframe showing the influence of each climate index as covariate in the nonstationary GEV model.

In [ ]:
stationID = station_ids[3]
df_cvteLocation = pd.read_json(Path(dirs['model_output_dir']) / str(stationID) / 'cvte_location_params_ALL.json')
df_cvteLocation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig,ax = plt.subplots(1,len(station_ids),figsize=(8,5))

for i,stationID in enumerate(station_ids):

    station_name = station_names[int(stationID)]
    # Create the full path for the JSON file
    jsonpath = Path(dirs['model_output_dir']) / str(stationID) / 'cvte_location_params_ALL.json'

     # Open and read the JSON file
    with open(jsonpath, 'r') as f:
        output = json.load(f)
    
    df_cvteLocation = pd.DataFrame(output)

    # Define colors based on significance
    location_colors = np.where(df_cvteLocation['Significance (over Nodal)']>0.95, 'green', 'lightgray')


    # Plot the Location parameter with custom colors for significance
    df_cvteLocation.plot(
        x='Climate Index',
        y='CI param',
        kind='barh',  # Horizontal bar plot
        ax=ax[i],
        color=location_colors
    )

    # add error bars
    ax[i].errorbar(df_cvteLocation['CI param'], df_cvteLocation.index, 
                   xerr=df_cvteLocation['Standard Error of CI param'], fmt='none', ecolor='black', capsize=0, lw =0.5)
    
    # add lag to each bar
    
    halign = 'left'
    for j in range(len(df_cvteLocation)):
        xoffset = 0.001
        if df_cvteLocation['CI param'].iloc[j]<0:
            xoffset = -df_cvteLocation['CI param'].iloc[j]+xoffset
        ax[i].text(df_cvteLocation['CI param'].iloc[j]+xoffset, j+0.2, '{:.0f}'.format(df_cvteLocation['lag'].iloc[j]), fontsize=6, ha=halign, va='center', color='black')


    ax[i].legend().set_visible(False)

    # Remove y-axis label
    ax[i].set_ylabel('')

    # remove x-tick labels
    if i != 0:
        ax[i].set_yticklabels([])
        ax[i].set_yticks([])

    # set all axes to the same scale
    ax[i].set_xlim(-0.05,0.05)

    # set xticks to have only 3 ticks
    ax[i].set_xticks(np.linspace(-0.05,0.05,3))

    # rotate xticks
    ax[i].tick_params(axis='x', rotation=45)

    # set fontsize of xticks
    ax[i].tick_params(axis='x', labelsize=6)
    ax[i].set_title(station_name.split(',')[0].strip() + '\n (' + str(stationID) + ')', fontsize=8)

    # add super title
    fig.suptitle('Climate Index Coefficient Magnitude in Location parameter', fontsize=10)

    # add y label to full plot
    ax[0].set_ylabel('Climate Index', fontsize=10)

    # add x label to full plot
    ax[3].set_xlabel('Coefficient magnitude (cm)', fontsize=10, labelpad=10)


# glue figure
glue("CI_mag_inLocation",fig,display=False)

plt.savefig(output_dir / 'CI_mag_location.png', dpi=300, bbox_inches='tight')




```{glue:figure} CI_mag_inLocation
:name: "CI_mag_inLocation"

A comparison of the magnitudes of the climate index covariate in the location parameter of the nonstationary GEV model. The tested climate indices are shown in green when their inclusion in the total model (after the inclusion of the nodal cycle) is considered a significant contribution. Coefficients which do not offer a significant improvement over the Nodal model are shown in gray. The lag (in months) is shown next to each bar plot. Note that for this run, we have held the PMM index at a 10 month lag ({cite:t}`long_higher_2020` uses 8-month lag for upper 100m), PDO at 16 months, and ONI and BEST at 18 months for the Hawaiian Island stations ({cite:t}`long_higher_2020` find a 19-month lag for Niño3). Midway (Sand Island) is considered to be far enough away from the Hawaiian Island chain to have its own lag. Note there are some stations with covariates that would probably be considered significant at different lags.
```

Note that here PDO, BEST, PMM and ONI all have the same lag if they are Hawaiian Island stations. This is a fixed lag based on average results across all stations and/or pre-existing literature. I'm doing this for consistency across the region. 

For the record, we can look at the original "best" lags from the original correlation output file:

In [ ]:
# Load data
dfLagsOriginal = pd.read_csv(Path(dirs['CI_dir']) / 'CI_correlation_results.csv')
dfLagsSet = pd.read_csv(Path(dirs['CI_dir']) / 'CI_correlation_results_setLag.csv')

CInames = ['PMM','PDO','BEST','ONI']

# make bar labels the right color
import matplotlib.patches as mpatches

# Define colors
auto_color = plt.rcParams['axes.prop_cycle'].by_key()['color'][0]  
set_color = plt.rcParams['axes.prop_cycle'].by_key()['color'][1]   

auto_patch = mpatches.Patch(color=auto_color, label='Automated Lag Finding')
set_patch = mpatches.Patch(color=set_color, label='Set Lags')

def barlabel(ax, bar, lag, yoffset=0.01):
    if bar.get_height() > 0:
        yoffset = 0.01
        valign = 'bottom'
    else:
        yoffset = -0.01
        valign = 'top'

    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + yoffset, f'{int(lag)}', 
                ha='center', va=valign, fontsize=8, color=bar.get_facecolor())
    

# Plot the bars side by side
fig, axes = plt.subplots(2, 2, figsize=(8, 5))

for i, CIname in enumerate(CInames):
    ax = axes.flatten()[i]
    CIname = CInames[i]

    dfLagsCI_1 = dfLagsOriginal[dfLagsOriginal['climateIndex'].isin([CIname])]
    dfLagsCI_2 = dfLagsSet[dfLagsSet['climateIndex'].isin([CIname])]

    # Define bar width
    width = 0.4  

    # Create numerical x positions for stations
    stations = dfLagsCI_1['station'].unique()
    x = np.arange(len(stations))  # Numeric x positions



    bars1 = ax.bar(x - width/2, dfLagsCI_1['max_corr'], width, label='Automated Lag Finding', align='center')
    bars2 = ax.bar(x + width/2, dfLagsCI_2['max_corr'], width, label='Set Lags', align='center')

    for j, bar in enumerate(bars1):
        barlabel(ax, bar, dfLagsCI_1['lag'].iloc[j])
        barlabel(ax, bars2[j], dfLagsCI_2['lag'].iloc[j])

    # make the bars transparent if they are not significant
    for j, bar in enumerate(bars1):
        if dfLagsCI_1['significant'].iloc[j] == 0:
            bar.set_color('lightgray')
    for j, bar in enumerate(bars2):
        if dfLagsCI_2['significant'].iloc[j] == 0:
            bar.set_color('lightgray')
                


    # Set x-axis labels to station names
    if i>1:
        ax.set_xticks(x)
        ax.set_xticklabels(stations, rotation=45, ha='right')
    else:
        # Remove x-axis labels
        ax.set_xticks([])
        ax.set_xticklabels([])

    if i==1:
        ax.legend(handles=[auto_patch, set_patch], loc='upper left', bbox_to_anchor=(1, 1))
    if i==0 or i==2:
        ax.set_ylabel('Correlation at Lag')
    else:
        ax.set_yticklabels([])
        ax.set_ylabel('')

    # Add labels and legend
    ax.set_title(CIname)

    # set y-axis limits
    ax.set_ylim(-0.3,0.5)


fig.suptitle('Correlation at Lag for Climate Indices', fontsize=12)

glue("CI_lag_correlation",fig,display=False)


```{glue:figure} CI_lag_correlation
:name: "CI_lag_correlation"

In this plot we examine the differences in correlation from climate index to monthly maxima, between holding the lag constant for the four climate indices shown, or allowing the algorithm to find the peak correlation automatically. For PMM, there is little change in correlation between using the set or auto lag. Likewise with PDO. 
```

## Plot all models in time
Next we'll look at all models and how they compare.

In [ ]:
# Only do Honolulu for now
stationID = station_ids[5]

modelTypes = ['RL_muT', 'RL_muN','RL_muT_cv1','RL_muT_cv2','RL_muT_N_cv1','RL_muT_N_cv2','RL_best']

for modelType in modelTypes:
    modelName = modelType + '.nc'
    modelPath = Path(dirs['model_output_dir'] / str(stationID) / modelName)

    if not modelPath.exists():
        continue
    model = xr.open_dataset(modelPath)

    model['ReturnLevel'].sel(ReturnPeriod=2).plot(label=modelType)

plt.legend()
# put the legend outside the plot
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# read the json file for best model
jsonpath = Path(dirs['model_output_dir']) / str(stationID) / 'best_params.json'
with open(jsonpath, 'r') as f:
    output = json.load(f)
    modelInfo = output['modelInfo']
    params = output['w']
    x = output['x']

time = modelInfo['year0'] + pd.Series(modelInfo['t'])
plt.scatter(time, modelInfo['monthlyMax'], color='k', label='Monthly Maxima', s=5)

plt.title('2-Year Return Levels: ' + model.attrs['station_name'])

w = helpers.adjust_w_for_plotting(x, params)

## Plot Components in time
First we'll make a function for plotting.

In [ ]:
def plot_location(w, modelInfo, label, offset=0, component=True):
    # Unpack the w array
    b0, b1, b2, b3, b4, b5, b6 = w[0], w[3], w[4], w[5], w[6], w[7], w[8]
    a0, bLT, bCI, aCI, bN1, bN2 = w[1], w[9], w[10], w[11], w[12], w[13]

    # Check if w[10] (bCI) is zero (i.e., no covariate contribution)
    if bCI == 0:
        ti = np.arange(0, modelInfo['t'][-1], 0.01)
        CI = 0  # Covariate is zero if bCI is zero
    else:
        ti = pd.Series(modelInfo['t']) 
        CI = pd.Series(modelInfo['covariate'])  # Covariate values

        

    # Make sure CI and ti are aligned (if CI is a series)
    if isinstance(CI, pd.Series) and len(CI) != len(ti):
        raise ValueError("Length of covariate CI does not match the length of ti.")

    # Define mut (location(t)) as a function of ti and CI
    mut = (b0 * np.exp(bLT * ti) +
           b1 * np.cos(2 * np.pi * ti) + b2 * np.sin(2 * np.pi * ti) +
           b3 * np.cos(4 * np.pi * ti) + b4 * np.sin(4 * np.pi * ti) +
           b5 * np.cos(8 * np.pi * ti) + b6 * np.sin(8 * np.pi * ti) +
           bN1 * np.cos((2 * np.pi / 18.61) * ti) + bN2 * np.sin((2 * np.pi / 18.61) * ti) +
           (bCI * CI))  

    # Calculate the amplitudes for each harmonic and the covariate
    amp_annual = np.sqrt(b1**2 + b2**2)      # First harmonic (annual cycle)
    amp_semiannual = np.sqrt(b3**2 + b4**2)  # Second harmonic (semiannual cycle)
    amp_third = np.sqrt(b5**2 + b6**2)       # Third harmonic (quarterly or 8π cycle)
    amp_nodal = np.sqrt(bN1**2 + bN2**2)     # Nodal component
    amp_covariate = abs(bCI)                 # Covariate amplitude
    amp_trend = 0.5*(b0 * np.exp(bLT * modelInfo['t'][-1]) - b0*np.exp(bLT * modelInfo['t'][0]))  # Trend amplitude



    amp_seasonal = np.sqrt(amp_annual**2 + amp_semiannual**2 + amp_third**2)
    
    # Calculate the total amplitude
    amp = np.sqrt(amp_seasonal**2 + amp_nodal**2 + amp_covariate**2 + amp_trend**2)
    


    # Subtract the datum offset
    mut = mut - modelInfo['STNDtoMHHW']

    if component:
        mut_demeaned = mut - np.mean(mut)
    else:
        mut_demeaned = mut
    
    # mut_demeaned = mut_demeaned - modelInfo['STNDtoMHHW']
    
    # add zero line for reference in gray
    plt.axhline(y=offset, color='gray', linestyle='-.', alpha=0.5)

    # Plot the result

    plt.plot(ti+modelInfo['year0'], mut_demeaned + offset, label=label + ' (' + str(round(2*amp*100, 2)) + ' cm)') 
    return amp



Then we'll make a function for making the figure.

In [ ]:
# Make figure
def make_component_figure(w, modelInfo, model, time):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    returnYear = 10

    # Add returnYear-year return level to the plot
    plt.plot(model['Year']+0.5, 
            model['ReturnLevel'].sel(ReturnPeriod=returnYear)-modelInfo['STNDtoMHHW'], 
            label=f'{returnYear}-year Return Level', color='black', linewidth=2) 

    # add error bars as shaded area from model['RL_low'] to model['RL_high']
    plt.fill_between(model['Year']+0.5, 
                    model['RL_high'].sel(ReturnPeriod=returnYear)-modelInfo['STNDtoMHHW'], 
                    model['RL_low'].sel(ReturnPeriod=returnYear)-modelInfo['STNDtoMHHW'],
                    color='gray', alpha=0.2)

    # Add monthly maxima, centered
    mm = np.array(modelInfo['monthlyMax']) - modelInfo['STNDtoMHHW']
    plt.scatter(time, mm, color='gray', label='Monthly Maxima', s=5)
    amp_full = plot_location(w, modelInfo, label='Full $\mu(t)$', offset=0, component=False)

    # First plot: Covariate contribution
    wnew = np.zeros(len(w))
    wnew[10] = w[10]  # Only use the covariate component
    offset1 = -0.2  # Offset for the plot
    amp_CI = plot_location(wnew, modelInfo, label=modelInfo['covariateName'] , offset=offset1)


    # Second plot: Seasonal components
    wnew = np.zeros(len(w))
    wnew[1:9] = w[1:9]  # Seasonal components
    offset1 -= 0.2  # Increment offset correctly
    amp_S = plot_location(wnew, modelInfo, label='Seasonal', offset=offset1)

    # Third plot: Long-term trend
    wnew = np.zeros(len(w))
    wnew[9] = w[9]  # Long-term trend component
    wnew[0] = w[0]  # Plus base value
    offset1 -= 0.2  # Increment offset correctly
    amp_LT = plot_location(wnew, modelInfo, label='Long-term trend', offset=offset1)


    # Fourth plot: Nodal component
    wnew = np.zeros(len(w))
    wnew[12:14] = w[12:14]  # Nodal components
    offset1 -= 0.2  # Increment offset correctly
    amp_N = plot_location(wnew, modelInfo, label='Nodal', offset=offset1)

    plt.title('Location Parameter Components: ' + model.attrs['station_name'])
    plt.xlim(modelInfo['t'][0] + modelInfo['year0'], modelInfo['t'][-1] + modelInfo['year0'])
    plt.ylabel('Height above MHHW (m)')

    # Add legend to the outside of the plot
    plt.legend(loc='upper left', bbox_to_anchor=(1, 1))
    plt.subplots_adjust(right=0.85)

    # add a second y-axis on the right side to label components
    ax2 = ax.twinx()
    ax2.set_ylim(ax.get_ylim())
    ax2.set_yticks(np.arange(0, offset1-0.2, -0.2))
    # ax2.set_yticklabels([' Full $\mu(t)$', ' ' +
    #                   modelInfo['covariateName'] + ' $\mu_{CI}(t)$' , 
    #                   ' Seasonal $\mu_{S}(t)$', 
    #                   ' Long-term trend $\mu_{LT}(t)$', 
    #                   ' Nodal $\mu_{N}(t)$']); #not sure why there's output here but I've suppressed it 
    ax2.set_yticklabels([' Full $\\mu(t)$', ' ' +
                     modelInfo['covariateName'] + ' $\\mu_{CI}(t)$' ,
                     ' Seasonal $\\mu_{S}(t)$',
                     ' Long-term trend $\\mu_{LT}(t)$',
                     ' Nodal $\\mu_{N}(t)$'])
    
    amps = [amp_full, amp_CI, amp_S, amp_LT, amp_N]
    amp_names = ['Full', 'Covariate', 'Seasonal', 'Long-term trend', 'Nodal']

        # Get the default color cycle
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key()['color']
    #x_line_position = ax.get_xlim()[1] + 0.5
    x_line_position = 0.86
    y_ticks = ax2.get_yticks()
    # Draw vertical lines on the figure using figure-relative coordinates
    for i, amp in enumerate(amps):
        ymin_fig = ax2.transData.transform((0, y_ticks[i] - amp))[1]  # Bottom of the line
        ymax_fig = ax2.transData.transform((0, y_ticks[i] + amp))[1]  # Top of the line
        ymin_norm = fig.transFigure.inverted().transform((0, ymin_fig))[1]  # Convert to figure normalized
        ymax_norm = fig.transFigure.inverted().transform((0, ymax_fig))[1]  # Convert to figure normalized
        
        # Plot vertical line in figure coordinates
        fig.lines.append(plt.Line2D([x_line_position, x_line_position], [ymin_norm, ymax_norm],
                                    transform=fig.transFigure, solid_capstyle='round',
                                    color = color_cycle[i],linestyle='-', linewidth=3))


    amps = dict(zip(amp_names, amps))
    return fig, amps, ax


Finally, we'll plot it up!

In [ ]:
for stationID in station_ids:

    # read the json file for best model
    jsonpath = Path(dirs['model_output_dir']) / str(stationID) / 'best_params.json'
    with open(jsonpath, 'r') as f:
        output = json.load(f)
        modelInfo = output['modelInfo']
        params = output['w']
        x = output['x']
        
    time = modelInfo['year0'] + pd.Series(modelInfo['t'])

    w = helpers.adjust_w_for_plotting(x, params)
    modelType = 'RL_best.nc'
    model = xr.open_dataset(Path(dirs['model_output_dir']) / str(stationID) / modelType)
    fig, amps, ax = make_component_figure(w, modelInfo, model, time)

    #save the figure
    savename = 'LocationComponents_' + model.attrs['station_name'] + '.png'
    filename = Path(dirs['output_dir']) / savename
    fig.savefig(filename, bbox_inches='tight')

    # save amps to a json file
    amps_json = Path(dirs['model_output_dir']) / str(stationID) / 'location_param_component_amps.json'
    with open(amps_json, 'w') as f:
        json.dump(amps, f)



Now we'll save the parameter component amplitudes to our model output directory.

In [ ]:
stationID = int(station_ids[3])

# read the json file for best model
jsonpath = Path(dirs['model_output_dir']) / str(stationID) / 'best_params.json'
with open(jsonpath, 'r') as f:
        output = json.load(f)
        modelInfo = output['modelInfo']
        params = output['w']
        x = output['x']
        
time = modelInfo['year0'] + pd.Series(modelInfo['t'])

w = helpers.adjust_w_for_plotting(x, params)
modelType = 'RL_best.nc'
model = xr.open_dataset(Path(dirs['model_output_dir']) / str(stationID) / modelType)
fig, amps, ax = make_component_figure(w, modelInfo, model, time)

glue("location_components",fig,display=False)
glue("station",model.attrs['station_name'],display=False)


```{glue:figure} location_components
:name: "location_components"

Location parameter components for the {glue:text}`station` tide gauge. 
```

## Plot return level curves 
We'll plot them using the 2023 return levels.

In [ ]:
import contextlib

def get_2000_2023_RL(best_params):
    """
    Calculate the return levels for the years 2023 and 2000 based on the best parameters.
    """
    ReturnPeriod = [0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50, 100]  # Example return periods in years
    years, RL, RL_high, RL_low = helpers.getTimeDependentReturnValue(
        best_params['modelInfo']['t'], 
        best_params['modelInfo']['covariate'], 
        best_params['w'], 
        best_params['x'], 
        ReturnPeriod, 
        best_params['mio']
    )
    year0 = best_params['modelInfo']['year0']
    years = years + year0

    # Find the index where years == 2023
    year_2023 = np.where(years == 2023)[0][0]
    year_2000 = np.where(years == 2000)[0][0]

    RLs2023 = RL[:, year_2023]
    RLs2000 = RL[:, year_2000]

    return RLs2023, RLs2000, ReturnPeriod



def get_amplitudes_and_tseries(STATION_ID,ReturnPeriod=[20]):
    """
    Get amplitudes and time series for a given station ID.
    
    Args:
        STATION_ID (str): The station ID to retrieve data for.
        
    Returns:
        tuple: A tuple containing the amplitudes and time series.
    """
    params_json = dirs['model_output_dir'] / str(STATION_ID) / 'best_params.json'

    if params_json.exists():
        print(f"Loading model parameters from: {params_json}")
        with open(params_json, 'r') as f:
            output = json.load(f)
            params = output['w']
            mio = output['mio']
            x = output['x']
            modelInfo = output['modelInfo']



        w_noLL = helpers.adjust_w_for_plotting(x, params)
        # insert arbitary log likelihood value at the start
        w = np.insert(w_noLL, 0, 400)
        w_best = w[w != 0]  # Remove any zero values

        x_seasonal = x[0:3]

        with contextlib.redirect_stdout(io.StringIO()):  # Suppress output
            # Calculate return levels for the best model
            years, RL_best, _, _ = helpers.getTimeDependentReturnValue(modelInfo['t'], modelInfo['covariate'], w_best, x, ReturnPeriod, mio)

        # Include only astronomical cycles
        w_nodal = w.copy()
        w_nodal = np.delete(w_nodal, [10, 11,12])

        # remove any zero values from w_nodal
        w_nodal = w_nodal[w_nodal != 0]
        x_nodal = x_seasonal + [0, 0, 0, 1]

        # if nodal is not included, remove it from x_nodal
        if x[6] == 0:
            x_nodal[6] = 0  # Ensure nodal component is zeroed out

        with contextlib.redirect_stdout(io.StringIO()):  # Suppress output
            years, RL_nodal, _, _ = helpers.getTimeDependentReturnValue(modelInfo['t'], modelInfo['covariate'], w_nodal, x_nodal, ReturnPeriod, mio)

        # Include only trend
        w_trend = w[0:11]
        if w_trend[10] != 0:
            w_trend = w_trend[w_trend != 0]  # Remove any zero values
            x_trend = x_seasonal + [1,0,0,0]
        else:
            w_trend = w_trend[w_trend != 0]  # Remove any zero values
            x_trend = x_seasonal + [0,0,0,0]

        with contextlib.redirect_stdout(io.StringIO()):  # Suppress output
            # Calculate return levels for the trend model
            years, RL_trend, _, _ = helpers.getTimeDependentReturnValue(modelInfo['t'], modelInfo['covariate'], w_trend, x_trend, ReturnPeriod, mio)

        # Include only MMA
        w_cvte = np.zeros_like(w)
        # Set w_cvte to match w up to index 13, but skip index 10
        w_cvte[:13] = w[:13]
        w_cvte[10] = 0  # Ensure index 10 is zeroed out
        w_cvte = w_cvte[w_cvte != 0]  # Remove any zero values

        # use x from original model, remove index 3 and 6 (trend and nodal)
        x_cvte = x.copy()
        x_cvte[3] = 0  # Remove trend component
        x_cvte[6] = 0  # Remove nodal component

        with contextlib.redirect_stdout(io.StringIO()):  # Suppress output
            # Calculate return levels for the MMA model
            years, RL_CI, _, _ = helpers.getTimeDependentReturnValue(modelInfo['t'], modelInfo['covariate'], w_cvte, x_cvte, ReturnPeriod, mio)

        # Seasonal Only model
        w_seasonal = w[:10]  # Use first 10 components (seasonal components
        w_seasonal = w_seasonal[w_seasonal != 0]  # Remove any zero values

        x_sea = x_seasonal + [0, 0, 0, 0]  # Use seasonal components only

        with contextlib.redirect_stdout(io.StringIO()):  # Suppress output
            # Calculate return levels for the seasonal model
            years, RL_seasonal,_,_ = helpers.getTimeDependentReturnValue(modelInfo['t'], None, w_seasonal, x_sea, ReturnPeriod, mio)


        # Create a clean DataFrame with all model variants
        df = pd.DataFrame({
            'years': years[:-1]+modelInfo['year0'],  # Remove last element to match RL arrays
            'RL_seasonal': RL_seasonal[0],  
            'RL_Trend': RL_trend[0], 
            'RL_Nodal': RL_nodal[0],
            'RL_CI': RL_CI[0],
            'RL_full': RL_best[0]
        })

        # zero out RL if not in the best model (x)
        if x[3] == 0:
            df['RL_Trend'] = 0.0
        if x[6] == 0:
            df['RL_Nodal'] = 0.0
        if x[4] == 0:
            df['RL_CI'] = 0.0

        # Add metadata
        df.attrs = {
            'return_period': ReturnPeriod[0],
            'station_id': modelInfo.get('recordID', 'Unknown'),
            'description': 'Time-dependent return levels by model component'
        }

        # print(f"DataFrame created with {len(df)} time points")
        # print(f"Columns: {list(df.columns)}")
        # print(f"Return Period: {ReturnPeriod[0]} years")

        # Calculate amplitude of each column, using  peak-to-peak amplitude
        amplitude = {
            'RL_Trend': 0.5*100*(df['RL_Trend'].iloc[-1] - df['RL_Trend'].iloc[0]),
            'RL_Nodal': 0.5*100*(df['RL_Nodal'].max() - df['RL_Nodal'].min()),
            'RL_CI': 0.5*100*(df['RL_CI'].max() - df['RL_CI'].min()),
            'RL_full': 0.5*100*(df['RL_full'].max() - df['RL_full'].min())
            }

    else:
        # make corresponding empty DataFrame
        df = pd.DataFrame(columns=['years', 'RL_seasonal', 'RL_Trend', 'RL_Nodal', 'RL_CI', 'RL_full'])
        df.attrs = {
            'return_period': ReturnPeriod[0],
            'station_id': STATION_ID,
            'description': 'Time-dependent return levels by model component'
        }
        amplitude = {
            'RL_Trend': 0.0,
            'RL_Nodal': 0.0,
            'RL_CI': 0.0,
            'RL_full': 0.0
        }   

    return df, amplitude

In [ ]:
def get_plot_data(STATION_ID):
    """
    Get plot data for a given station ID.
    
    Args:
        STATION_ID (str): The station ID to retrieve data for.
        
    Returns:
        dict: A dictionary containing RL_best, best_params, df, amplitude, RLs2000, RLs2023, ReturnPeriod.
    """
    
    # open RL_best.nc
    RL_best = xr.open_dataset(f"{dirs['model_output_dir']}/{STATION_ID}/RL_best.nc")

    # open best_params.json
    with open(f"{dirs['model_output_dir']}/{STATION_ID}/best_params.json", 'r') as f:
        best_params = json.load(f)

    df, amplitude = get_amplitudes_and_tseries(STATION_ID, ReturnPeriod=[20])
    RLs2023, RLs2000, ReturnPeriod = get_2000_2023_RL(best_params)
    mhhw = best_params['modelInfo']['STNDtoMHHW']

    # make a dict of RL_best, best_params, df, amplitude, RLs2000, RLs2023, ReturnPeriod
    plot_data = {
        'RL_best': RL_best,
        'best_params': best_params,
        'df': df,
        'amplitude': amplitude,
        'RLs2023': RLs2023,
        'RLs2000': RLs2000,
        'ReturnPeriod': ReturnPeriod,
        'mhhw': mhhw
    }
    return plot_data


In [ ]:
def plot_return_level_curves(ax, RLs2023, RLs2000, ReturnPeriod, mhhw, amplitude, best_params,df):
    """
    Plot return level curves for a station on the given axis.
    Args:
        ax (matplotlib.axes.Axes): Axis to plot on.
        RLs2023 (array): Return levels for 2023.
        RLs2000 (array): Return levels for 2000.
        ReturnPeriod (array): Return periods (years).
        mhhw (float): Mean higher high water reference.
        amplitude (dict): Amplitudes for each model component.
        best_params (dict): Model parameters and metadata.
    """
    # Plot return levels for 2023 and 2000
    ax.plot(ReturnPeriod, RLs2023-mhhw, label='Return Level 2023', color='k')
    ax.plot(ReturnPeriod, RLs2000-mhhw, label='Return Level 2000', color='k', linestyle='--')
    covariateName = best_params.get('modelInfo', {}).get('covariateName', 'Covariate')
    CIlabel = r'{} Contribution ($\pm$ {:.1f} cm)'.format(covariateName, amplitude["RL_CI"])

    # add an offset, which actual amplitude of the CI effect in 2023
    offset = df[df['years'] == 2023]['RL_CI'] - df[df['years'] == 2023]['RL_seasonal']
    offset = offset.values[0]
    offset = 0

    ax.fill_between(ReturnPeriod, RLs2023-mhhw+offset+0.01*amplitude['RL_CI'],
                    RLs2023-mhhw+offset-0.01*amplitude['RL_CI'], color='gray', alpha=0.2, label=CIlabel)

    # # Annotate amplitudes if available
    # if amplitude:
    #     for key, amp in amplitude.items():
    #         ax.text(0.98, 0.02 + 0.04 * list(amplitude.keys()).index(key), f'{key}: {amp:.1f} cm',
    #                 transform=ax.transAxes, fontsize=8, color='gray', ha='right', va='bottom')
    # Set log scale for x-axis (return period)
    ax.set_xscale('log')

    # make x-axis labels be 0.1, 1, 10, 100 years
    ax.set_xticks([0.1, 1, 10, 100])
    ax.get_xaxis().set_major_formatter(plt.ScalarFormatter())

    # set x-axis limits
    ax.set_xlim(0.1, 100)

    # add grid
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)

    ax.set_xlabel('Return Period (years)')
    ax.set_ylabel('Return Level (m, MHHW)')


    # Title with station ID
    station_id = best_params.get('modelInfo', {}).get('stationID', 'Unknown')
    station_name = best_params.get('modelInfo', {}).get('station_name', 'Unknown Station')
    ax.set_title(f'Return Level Curves for {station_name} ({station_id})')
    ax.legend()

In [ ]:
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(1, 1, 1)
plot_data = get_plot_data(stationID)
plot_return_level_curves(ax, plot_data['RLs2023'], plot_data['RLs2000'], 
                         plot_data['ReturnPeriod'], plot_data['mhhw'],
                         plot_data['amplitude'], plot_data['best_params'],plot_data['df'])

#glue 
glue("return_level_curves_NS",fig,display=False)

# save figure
savepath = output_dir / f'ReturnLevelCurves_NS_{stationID}.png'
fig.savefig(savepath, bbox_inches='tight')

In [ ]:
covariateName = plot_data['best_params'].get('modelInfo', {}).get('covariateName', 'Covariate')
amplitudeCI = plot_data['amplitude'].get('RL_CI', 0)

glue("covariateName",covariateName,display=False)
glue("ampCI",amplitudeCI,display=False)

```{glue:figure} return_level_curves_NS
:name: "return_level_curves_NS"

Return level curves in 2023 and 2000 for the {glue:text}`station` tide gauge. The shaded area represents the influence of the {glue:text}`covariateName` covariate in the return levels, which can influence the location parameter by $\pm$ {glue:text}`ampCI:.2f` cm.
```

## Plot components by station

The following code can only be run after all gauges have been modeled!

In [ ]:
# get all location_parms_component_amp.json files and combine them into one
amps = {}

for stationID in station_ids:
    amps_json = Path(dirs['model_output_dir']) / str(stationID) / 'location_param_component_amps.json'
    with open(amps_json, 'r') as f:
        amps[stationID] = json.load(f)


In [ ]:
# Plot df as stacked bar chart ignoring the full component
# Turn amps into a dataframe
df = pd.DataFrame(amps).T
df.drop('Full', axis=1,inplace=True)

# Reorder columns to match above plot
df = df[['Covariate','Seasonal', 'Long-term trend',  'Nodal']]
#rename 'Covariate' to 'BEST (19-month lag)'
# df = df.rename(columns={'Covariate':'BEST (19-month lag)'})
df = df.sort_index()

# conver the data colums to cm and round to 2 decimal places
df = df.apply(lambda x: x*100).round(2)

# add station names to the dataframe
df['Station Name'] = station_names.values()


# Plot the data with amplitudes on the x-axis
df.plot(kind='barh', stacked=True, figsize=(10, 6), color=plt.cm.tab10.colors[1:])
plt.title('Location Parameter Component Amplitudes')
plt.xlabel('Amplitude (cm)')
plt.legend(title='Component', loc='upper left', bbox_to_anchor=(1, 1))

# replace the x-axis with the station names
plt.yticks(np.arange(len(df.index)), df['Station Name']);

# save the plot
savename = Path(dirs['output_dir']) / 'location_param_component_amps.png'
plt.savefig(savename, bbox_inches='tight')

glue("location_param_component_amps",plt.gcf(),display=False)


```{glue:figure} location_param_component_amps
:name: "location_param_component_amps"
Bar plot showing the amplitude of individual covariate contributions (seasonal, long-term trend, climate variability and nodal tidal cycle) to sea level in the location parameter. The amplitude of each factor is displayed in centimeters. The covariate shown here corresponds to the best fit climate covariate to the nonstationary GEV model.

```

## Plot a map of covariate contributions
Here we'll look at the influence of the climate covariate in the form of a map. First we'll extract the amplitudes of the covariate from every station we've analyzed.


In [ ]:
# load rsl data
with xr.open_dataset(dirs['data_dir'] / 'rsl_daily_hawaii.nc') as rsl:
    df = rsl[['lat', 'lon', 'station_name', 'station_id']].to_dataframe().reset_index()
    pass

# Initialize ampCI as a vector of zeros
df['ampCI'] = 0

# Assign ampCI values using the station_id column with vectorization, where possible
df['ampCI'] = df['station_id'].map(lambda sid: amps.get(sid, {}).get('Covariate', 0))

# Initialize modelCI with 'None' as default
df['modelCI'] = 'None'

# Function to get modelCI, with improved error handling
def get_model_ci(station_id):
    fpath = Path(dirs['model_output_dir']) / str(station_id) / 'RL_best.nc'
    try:
        with xr.open_dataset(fpath) as model:
            return model.attrs.get('covariate', 'None')
    except FileNotFoundError:
        print(f'No model found for stationID: {station_id}, skipping...')
        return 'None'
    except Exception as e:
        print(f'Error processing {station_id}: {e}')
        return 'None'

# Apply the function to the DataFrame (efficient vectorized operation)
df['modelCI'] = df['station_id'].apply(get_model_ci)


Next we'll plot up our map.

In [ ]:
from matplotlib import colors, cm
from maps import plot_thin_map_hawaii, get_stationinfo
from matplotlib.lines import Line2D

In [ ]:
df['ampCI'] = 100*df['ampCI'] # put ampCI in centimeters
df['ampCI'] = df['ampCI'].round(2)

# make all ampCI values if zero 0.05
df['ampCI'] = df['ampCI'].replace(0, 0.3)

In [ ]:

# Separate the "None" entries from the rest of the data
df_none = df[df['modelCI'] == 'None']  # Entries where modelCI is 'None'
df_filtered = df[df['modelCI'] != 'None']  # All other entries

# Use tab10 colormap for colors (excluding 'None')
cmap = plt.get_cmap('Set1')
color_dict = {index: cmap(i) for i, index in enumerate(np.unique(df_filtered['modelCI']))}

# Map the climate indices to numerical values for the colorbar
index_mapping = {index: i for i, index in enumerate(np.unique(df_filtered['modelCI']))}
mapped_indices = [index_mapping[ci] for ci in df_filtered['modelCI']]

# Create a colormap without "None"
norm = colors.BoundaryNorm(boundaries=np.arange(len(np.unique(df_filtered['modelCI'])) + 1) - 0.5, 
                               ncolors=len(np.unique(df_filtered['modelCI'])))

fig, ax = plt.subplots(1, 1, figsize=(10, 10), subplot_kw={'projection': ccrs.PlateCarree()})

# Call plot_thin_map_hawaii to create the figure and axis
plot_thin_map_hawaii(ax)

sizefac = 50
# Plot each station with varying color (based on climate index) and size (based on amplitude)
scatter = ax.scatter(df_filtered['lon'], df_filtered['lat'], 
                     c=mapped_indices, cmap=cmap, 
                     norm=norm, s=sizefac * df_filtered['ampCI'],
                     edgecolor='black', linewidth=0.5, transform=ccrs.PlateCarree())

# Plot the "None" stations in gray (fixed color)
ax.scatter(df_none['lon'], df_none['lat'], 
           color='gray', s=sizefac * df_none['ampCI'], 
           edgecolor='black', linewidth=0.5, transform=ccrs.PlateCarree())

dfInfo = get_stationinfo(dirs['data_dir'])
dfInfo['station_id'] = dfInfo['station_id']

#change the font color to black if the station is in the df_filtered
dfInfo['fontcolor'] = np.where(dfInfo['station_id'].isin(df_filtered['station_id']), 'black', 'gray')

# Add an axes for the colorbar
cax = fig.add_axes([0.13, 0.21, 0.76, 0.02])  

# dfInfo
# Add text labels with adjusted offsets and horizontal alignment
for i, name in enumerate(dfInfo['station_name']):
    ax.text(dfInfo['lon'][i] + dfInfo['offsetlon'][i], dfInfo['lat'][i] + dfInfo['offsetlat'][i], name, 
            ha=dfInfo['ha'][i], fontsize=10, transform=ccrs.PlateCarree(),rotation=0, color = dfInfo['fontcolor'][i])

# Add a colorbar for the climate index
cbar = plt.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax, orientation='horizontal')
cbar.set_ticks(np.arange(len(np.unique(df_filtered['modelCI']))))
cbar.set_ticklabels(np.unique(df_filtered['modelCI']))
cbar.set_label('Climate Index')


# Amplitude size legend
sizes = [50, 100, 150]
size_labels = ["0.5 cm", "1.5 cm", "3.0 cm"]  
size_legend_elements = [Line2D([0], [0], marker='o', color='none', label=label, 
                               markerfacecolor='gray', markersize=size / 10) 
                        for size, label in zip(sizes, size_labels)]
ax.legend(handles=size_legend_elements, title="Amplitude Size", loc="lower left")

# change axes limits to ensure all stations are visible
ax.set_ylim(15, 30)
ax.set_xlim(-180, -153)


# save the figure
savename = Path(dirs['output_dir']) / 'SL_contributions_map.png'
plt.savefig(savename, dpi=300, bbox_inches='tight')

glue("CI_map",fig,display=False)


```{glue:figure} CI_map
:name: "CI_map"

Map of dominant climate indices in the Hawaiian Islands region. The size of each circle at a given tide gauge represents the amplitude of the climate covariate in the location parameter of nonstationary extremes. ENSO-type climate indices are are well-represented with amplitudes between 1.5-3 cm. Here, the lags for ENSO-type indices in the main Hawaiian Islands are set at 18 months, which aligns well with previous literature (e.g. Long et al 2020). 
```

## Plot a map of more GEV components

In [ ]:
#turn station_names into a dict
station_names_dict = {station_id: name for station_id, name in zip(station_ids, station_names)}

In [ ]:
# make a csv of the parameters in the best model, using best_params.json
best_params_list = []

for station_id in station_ids:  
    params_file = model_output_dir / str(station_id) / 'best_params.json'
    if params_file.exists():
        with open(params_file, 'r') as f:
            best_params = json.load(f)

            wbest = adjust_w_for_plotting(best_params['x'], best_params['w'])

            b0, a0, xi, b1, b2, b3, b4, b5, b6 = wbest[:9] # basics + seasonal cycle
            bLT, bCI, aCI, bN1, bN2 = wbest[9:]  # added fun

        best_params_list.append({
            'station_id': station_id,
            'station_name': station_names_dict[station_id],
            'latitude': rsl_hourly.sel(station_id = station_id).lat.item(),
            'longitude': rsl_hourly.sel(station_id = station_id).lon.item(),
            'mhhw': best_params['modelInfo']['STNDtoMHHW'],
            'cvte': best_params['modelInfo']['covariateName'],
            'b0': b0,
            'a0': a0,
            'xi': xi,
            'b1': b1,
            'b2': b2,
            'b3': b3,
            'b4': b4,
            'b5': b5,
            'b6': b6,
            'bLT': bLT,
            'bCI': bCI,
            'aCI': aCI,
            'bN1': bN1,
            'bN2': bN2
        })

In [ ]:
# turn best_params_list into a dataframe
best_params_df = pd.DataFrame(best_params_list)
best_params_df

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
fig, axs = plt.subplots(2,2, figsize=(10,6), subplot_kw={'projection': ccrs.PlateCarree(central_longitude=-180)})

axs = axs.flatten()

params_to_plot = ['b0', 'a0', 'xi', 'bCI']

for i, param in enumerate(params_to_plot):
    ax = axs[i]

    paramplot = best_params_df[param].astype(float)
    if i==0:
        # For b0, we want to plot it relative to the MHHW
        paramplot = paramplot - best_params_df['mhhw'].astype(float)

    if param != 'bCI':
        sc = ax.scatter(best_params_df['longitude'], best_params_df['latitude'],
                       c=paramplot,
                       cmap='jet', s=100, edgecolor=None, alpha=0.7, transform=ccrs.PlateCarree())
        
        cb_ax = inset_axes(ax, width='40%', height='5%', loc='lower center', borderpad=2.5)
        plt.colorbar(sc, cax=cb_ax, orientation='horizontal')

    if param == 'bCI':
        marker_dict = {'PMM': 'o', 'BEST': 's', 'ONI': '^', 'PDO': 'p', 'AO': 'v', 'PNA': 'd', 'TNA': '*'}
        legend_handles = []
        scatters = []
    
        # Get all bCI values for color normalization
        all_bCI = best_params_df['bCI'].astype(float)
        vmin, vmax = all_bCI.min(), all_bCI.max()
    
        for covariate, marker in marker_dict.items():
            subset = best_params_df[best_params_df['cvte'] == covariate]
            if not subset.empty:
                sc = ax.scatter(
                    subset['longitude'], subset['latitude'],
                    c=subset['bCI'].astype(float),
                    cmap='jet', s=100, marker=marker, edgecolor=None, alpha=0.7,
                    vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree(),
                    label=covariate
                )
                scatters.append(sc)
                legend_handles.append(
                    plt.Line2D([0], [0], marker=marker, color='w', label=covariate,
                               markerfacecolor='gray', markeredgecolor='black', markersize=10)
                )
    
        ax.legend(handles=legend_handles, title='Covariate', loc='lower left', fontsize=8)
        cb_ax = inset_axes(ax, width='40%', height='5%', loc='lower center', borderpad=2.5)
        # Use the first scatter for the colorbar (all use same vmin/vmax/cmap)
        plt.colorbar(scatters[0], cax=cb_ax, orientation='horizontal')

    ax.set_title(param)
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.set_extent([-200, -60, 0, 68], crs=ccrs.PlateCarree())
    
    # plt.colorbar(sc, cax=cb_ax, orientation='horizontal')

for ax in axs:
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.set_extent([-180, -152, 15,30], crs=ccrs.PlateCarree())
    ax.set_xticks(np.arange(-180, -152, 5), crs=ccrs.PlateCarree())
    ax.set_yticks(np.arange(15,30,5), crs=ccrs.PlateCarree())
    ax.set_xticklabels(np.arange(-180, -152, 5), fontsize=8)
    ax.set_yticklabels(np.arange(15,30,5), fontsize=8)

axs[0].set_title ('Mean location parameter (b0) \n ~Average Monthly Max (rel. to MHHW)')
axs[1].set_title ('Scale parameter (a0) \n ~$\delta$ of Monthly Max')
axs[2].set_title ('Shape parameter (xi)')
axs[3].set_title ('Covariate Contribution (bCI) \n (rel. to normalized cvte signal)')
# save figure
fig.savefig(output_dir / 'best_params_map.png', dpi=300, bbox_inches='tight') 

glue("best_params_map",fig,display=False)


```{glue:figure} best_params_map
:name: "Best Parameters Map"
Map of the nonstationary GEV-based extreme water level return values for Hawaiian Island region stations. Year 2020 is shown here. Each nonstationary model includes some combination of seasonality, a long-term trend, and the 18.6 nodal-cycle, provided each parameter improves the significance of the GEV fit. No covariates are used for these particular best-fit models.
```

## Plot sub-annual contributions


In [ ]:
stationID = int(station_ids[3])
jsonpath = Path(dirs['model_output_dir']) / str(stationID) / 'best_params.json'
with open(jsonpath, 'r') as f:
    output = json.load(f)
    modelInfo = output['modelInfo']
    params = output['w']
    x = output['x']

w = helpers.adjust_w_for_plotting(x, params)    

# params should be a dictionary with the following
paramNames = ['b0','a0','g0','b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'bLT', 'bCI', 'cCI', 'bN1', 'bN2']
params = {paramNames[i]: w[i] for i in range(len(paramNames))}

t = np.array(modelInfo['t'])
ci = np.array(modelInfo['covariate'])
# Convert t to years
tyears = t + modelInfo['year0']

# Constants
api = np.pi # pi
ak = 2 * api # 2pi
ak18 = 2 * api / 18.61  # 18.61-year nodal cycle

# Calculate location parameter mu(t)
mut = (params['b0'] * np.exp(params['bLT'] * t) +
        params['b1'] * np.cos(ak * t) + params['b2'] * np.sin(ak * t) +
        params['b3'] * np.cos(2 * ak * t) + params['b4'] * np.sin(2 * ak * t) +
        params['b5'] * np.cos(4 * ak * t) + params['b6'] * np.sin(4 * ak * t) +
        params['bN1'] * np.cos(ak18 * t) + params['bN2'] * np.sin(ak18 * t) +
        params['bCI'] * ci)

# Adjust mu(t) to be relative to MHHW
mut = mut - modelInfo['STNDtoMHHW']


import calendar
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

# Create a colormap based on the number of unique years we're going to plot
t00 = np.arange(1993,2023)
# make colormap the same length as t00
cmap = plt.cm.Blues(np.linspace(0,1,len(t00)))

# for each year, plot mu(t) for that year, and color it by time based on the colormap
for t0 in t00:
    inYear = np.floor(tyears).astype(int) == t0
    color_index = np.where(t00 == t0)[0][0]  # Find the index of t0 in t00
    plt.plot(np.arange(0, 1, 1/12), mut[inYear], label=str(t0), color=cmap[color_index])


# annotate and make it pretty
plt.xlim(0,1-1/12)
# set the xticks to be the months
plt.xticks(np.arange(0, 1, 1/12), [calendar.month_abbr[i] for i in range(1, 13)])
plt.xlabel('Month')
plt.ylabel('$\mu_t$, relative to MHHW (m)')
plt.title('Location Parameter by Year: ' + modelInfo['station_name'])

# add a colorbar
sm = plt.cm.ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(vmin=t00.min(), vmax=t00.max()))
cbar = plt.colorbar(sm, ax=plt.gca(), orientation='vertical', pad=0.02)
cbar.set_label('Year')

glue("location_by_year",fig,display=False)
savename = 'location_by_year_' + modelInfo['station_name'] + '.png'
plt.savefig(output_dir / savename, dpi=300, bbox_inches='tight')

#glue the CI name
glue("CI_name",modelInfo['covariateName'],display=False)
glue("station",modelInfo['station_name'],display=False)

# get lag for covariateName
df_cvteLocation = pd.read_json(Path(dirs['model_output_dir']) / str(stationID) / 'cvte_location_params_ALL.json')

CI_lag = df_cvteLocation[df_cvteLocation['Climate Index'] == modelInfo['covariateName']]['lag'].values[0]
glue("CI_lag",int(CI_lag),display=False)

```{glue:figure} location_by_year
:name: "location_by_year"

This plot shows the changes in the location parameter (with covariate included) from 1993 through 2023 for the non-stationary GEV model at {glue:text}`station`. This particular calculation uses the {glue:text}`CI_name` climate index at a {glue:text}`CI_lag`-month lag. While there is a steady trend (background sea level rise), this is also affected by the covariate and nodal influences.
```


---

## Citations

```{bibliography}
:style: plain
:filter: docname in docnames
```